# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 7 2022 2:55PM,250216,10,8745767,"Snap Medical Industries, LLC",Released
1,Nov 7 2022 2:55PM,250216,10,8745768,"Snap Medical Industries, LLC",Released
2,Nov 7 2022 2:55PM,250216,10,8745769,"Snap Medical Industries, LLC",Released
3,Nov 7 2022 2:55PM,250216,10,8745770,"Snap Medical Industries, LLC",Released
4,Nov 7 2022 2:55PM,250216,10,SNAP1348673,"Snap Medical Industries, LLC",Released
5,Nov 7 2022 2:55PM,250216,10,SNAP32864666,"Snap Medical Industries, LLC",Released
6,Nov 7 2022 2:55PM,250216,10,SNAP32864710,"Snap Medical Industries, LLC",Released
7,Nov 7 2022 2:50PM,250212,10,8691190-BO,Eywa Pharma Inc.,Released
8,Nov 7 2022 2:50PM,250212,10,8691188-BO,Eywa Pharma Inc.,Released
9,Nov 7 2022 2:50PM,250212,10,8692269-BO,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,250207,Released,2
47,250211,Released,5
48,250212,Released,20
49,250214,Released,1
50,250216,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250207,NaN,NaN,2.0
250211,NaN,NaN,5.0
250212,NaN,NaN,20.0
250214,NaN,NaN,1.0
250216,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250118,5.0,1.0,0.0
250121,0.0,1.0,0.0
250130,0.0,0.0,1.0
250131,0.0,0.0,1.0
250133,1.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250118,5,1,0
250121,0,1,0
250130,0,0,1
250131,0,0,1
250133,1,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250118,5,1,0
1,250121,0,1,0
2,250130,0,0,1
3,250131,0,0,1
4,250133,1,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250118,5,1,
1,250121,,1,
2,250130,,,1
3,250131,,,1
4,250133,1,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 7 2022 2:55PM,250216,10,"Snap Medical Industries, LLC"
7,Nov 7 2022 2:50PM,250212,10,Eywa Pharma Inc.
27,Nov 7 2022 2:49PM,250211,10,Eywa Pharma Inc.
32,Nov 7 2022 2:46PM,250214,19,Methapharm-G
33,Nov 7 2022 2:29PM,250207,16,Sartorius Bioprocess Solutions
35,Nov 7 2022 2:27PM,250206,21,"NBTY Global, Inc."
36,Nov 7 2022 2:26PM,250204,21,"NBTY Global, Inc."
37,Nov 7 2022 2:21PM,250202,22,"NBTY Global, Inc."
38,Nov 7 2022 2:01PM,250184,20,"Emersa Waterbox, LLC"
39,Nov 7 2022 1:50PM,250201,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 7 2022 2:55PM,250216,10,"Snap Medical Industries, LLC",,,7
1,Nov 7 2022 2:50PM,250212,10,Eywa Pharma Inc.,,,20
2,Nov 7 2022 2:49PM,250211,10,Eywa Pharma Inc.,,,5
3,Nov 7 2022 2:46PM,250214,19,Methapharm-G,,,1
4,Nov 7 2022 2:29PM,250207,16,Sartorius Bioprocess Solutions,,,2
5,Nov 7 2022 2:27PM,250206,21,"NBTY Global, Inc.",,,1
6,Nov 7 2022 2:26PM,250204,21,"NBTY Global, Inc.",,,1
7,Nov 7 2022 2:21PM,250202,22,"NBTY Global, Inc.",,,1
8,Nov 7 2022 2:01PM,250184,20,"Emersa Waterbox, LLC",,,1
9,Nov 7 2022 1:50PM,250201,19,"GCH Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 7 2022 2:55PM,250216,10,"Snap Medical Industries, LLC",7,,
1,Nov 7 2022 2:50PM,250212,10,Eywa Pharma Inc.,20,,
2,Nov 7 2022 2:49PM,250211,10,Eywa Pharma Inc.,5,,
3,Nov 7 2022 2:46PM,250214,19,Methapharm-G,1,,
4,Nov 7 2022 2:29PM,250207,16,Sartorius Bioprocess Solutions,2,,
5,Nov 7 2022 2:27PM,250206,21,"NBTY Global, Inc.",1,,
6,Nov 7 2022 2:26PM,250204,21,"NBTY Global, Inc.",1,,
7,Nov 7 2022 2:21PM,250202,22,"NBTY Global, Inc.",1,,
8,Nov 7 2022 2:01PM,250184,20,"Emersa Waterbox, LLC",1,,
9,Nov 7 2022 1:50PM,250201,19,"GCH Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 7 2022 2:55PM,250216,10,"Snap Medical Industries, LLC",7,,
1,Nov 7 2022 2:50PM,250212,10,Eywa Pharma Inc.,20,,
2,Nov 7 2022 2:49PM,250211,10,Eywa Pharma Inc.,5,,
3,Nov 7 2022 2:46PM,250214,19,Methapharm-G,1,,
4,Nov 7 2022 2:29PM,250207,16,Sartorius Bioprocess Solutions,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 7 2022 2:55PM,250216,10,"Snap Medical Industries, LLC",7.0,NaN,NaN
1,Nov 7 2022 2:50PM,250212,10,Eywa Pharma Inc.,20.0,NaN,NaN
2,Nov 7 2022 2:49PM,250211,10,Eywa Pharma Inc.,5.0,NaN,NaN
3,Nov 7 2022 2:46PM,250214,19,Methapharm-G,1.0,NaN,NaN
4,Nov 7 2022 2:29PM,250207,16,Sartorius Bioprocess Solutions,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 7 2022 2:55PM,250216,10,"Snap Medical Industries, LLC",7.0,0.0,0.0
1,Nov 7 2022 2:50PM,250212,10,Eywa Pharma Inc.,20.0,0.0,0.0
2,Nov 7 2022 2:49PM,250211,10,Eywa Pharma Inc.,5.0,0.0,0.0
3,Nov 7 2022 2:46PM,250214,19,Methapharm-G,1.0,0.0,0.0
4,Nov 7 2022 2:29PM,250207,16,Sartorius Bioprocess Solutions,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2501824,47.0,20.0,15.0
15,1250792,121.0,22.0,37.0
16,1000654,4.0,1.0,0.0
18,250131,1.0,0.0,0.0
19,2251614,19.0,6.0,5.0
20,250184,1.0,0.0,0.0
21,1501001,6.0,0.0,0.0
22,1000772,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2501824,47.0,20.0,15.0
1,15,1250792,121.0,22.0,37.0
2,16,1000654,4.0,1.0,0.0
3,18,250131,1.0,0.0,0.0
4,19,2251614,19.0,6.0,5.0
5,20,250184,1.0,0.0,0.0
6,21,1501001,6.0,0.0,0.0
7,22,1000772,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,47.0,20.0,15.0
1,15,121.0,22.0,37.0
2,16,4.0,1.0,0.0
3,18,1.0,0.0,0.0
4,19,19.0,6.0,5.0
5,20,1.0,0.0,0.0
6,21,6.0,0.0,0.0
7,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,47.0
1,15,Released,121.0
2,16,Released,4.0
3,18,Released,1.0
4,19,Released,19.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,15.0,37.0,0.0,0.0,5.0,0.0,0.0,0.0
Executing,20.0,22.0,1.0,0.0,6.0,0.0,0.0,0.0
Released,47.0,121.0,4.0,1.0,19.0,1.0,6.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,15.0,37.0,0.0,0.0,5.0,0.0,0.0,0.0
1,Executing,20.0,22.0,1.0,0.0,6.0,0.0,0.0,0.0
2,Released,47.0,121.0,4.0,1.0,19.0,1.0,6.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,15.0,37.0,0.0,0.0,5.0,0.0,0.0,0.0
1,Executing,20.0,22.0,1.0,0.0,6.0,0.0,0.0,0.0
2,Released,47.0,121.0,4.0,1.0,19.0,1.0,6.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()